In [1]:
from useful_functions import *
import warnings
warnings.filterwarnings('ignore')
from category_encoders import LeaveOneOutEncoder,WOEEncoder
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(palette = "Dark2")
my_colors = [(0.10588235294117647, 0.6196078431372549, 0.4666666666666667),
  (0.8509803921568627, 0.37254901960784315, 0.00784313725490196)]
pd.set_option('display.max_columns', None)
from itertools import chain, combinations
import math


In [2]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_absolute_error
from catboost import CatBoostClassifier, Pool
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
import tensorflow as tensorflow
from lightgbm import LGBMClassifier
from sklearn.preprocessing import StandardScaler

### **5 Leave-one-out Encoding**
In order not to split the data into three different sets, we will use a leave-one-out encoding. This type of encoding calculates the target response mean for each variable **excluding** the current observation to reduce the effect of outliers. It further adds random noise to the target response to avoid overfitting.

In [34]:
df = pd.read_csv("df_processed.csv")
df

,order_item_id,order_date,delivery_date,item_id,size,item_color,brand_id,item_price,user_id,user_title,user_dob,user_state,user_reg_date,return,delivery_time,order_id,user_age,user_reg_age,order_weekday,delivery_weekday,order_month,delivery_month,order_day,delivery_day,order_week,delivery_week,order_item_count,order_sum,average_item_price_order,order_number_same_item_id,order_number_different_item_id,order_number_same_size,order_number_different_size,order_number_same_item_color,order_number_different_item_color,order_number_same_brand_id,order_number_different_brand_id,order_number_same_item_id_size,order_number_different_item_id_size,order_number_same_item_id_item_color,order_number_different_item_id_item_color,order_number_same_size_item_color,order_number_different_size_item_color,order_number_same_size_brand_id,order_number_different_size_brand_id,order_number_same_item_color_brand_id,order_number_different_item_color_brand_id,order_number_same_item_id_size_item_color,order_number_different_item_id_size_item_color,order_number_same_size_item_color_brand_id,order_number_different_size_item_color_brand_id
0,1,2012-04-01,2012-04-03,186,s,denim,25,69.90,794,Mrs,1965-01-06,Bad-Wue,2011-04-25,0.0,2,2012-04-01_794,47,342,6,1,4,4,1,3,13,14,3,209.8,69.93,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2
1,2,2012-04-01,2012-04-03,71,unsized,ocher,21,69.95,794,Mrs,1965-01-06,Bad-Wue,2011-04-25,1.0,2,2012-04-01_794,47,342,6,1,4,4,1,3,13,14,3,209.8,69.93,2,1,2,1,1,2,2,1,2,1,1,2,1,2,2,1,1,2,1,2,1,2
2,3,2012-04-01,2012-04-03,71,unsized,curry,21,69.95,794,Mrs,1965-01-06,Bad-Wue,2011-04-25,1.0,2,2012-04-01_794,47,342,6,1,4,4,1,3,13,14,3,209.8,69.93,2,1,2,1,1,2,2,1,2,1,1,2,1,2,2,1,1,2,1,2,1,2
3,4,2012-04-02,2012-04-06,22,s,green,14,39.90,808,Mrs,1959-11-09,Saxony,2012-01-04,0.0,4,2012-04-02_808,52,89,0,4,4,4,2,6,14,14,1,39.9,39.90,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0
4,5,2012-04-02,2012-04-06,151,s,black,53,29.90,825,Mrs,1964-07-11,S-Holstein,2011-02-16,0.0,4,2012-04-02_825,48,411,0,4,4,4,2,6,14,14,3,249.7,83.23,1,2,2,1,2,1,1,2,1,2,1,2,2,1,1,2,1,2,1,2,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531165,50074,2013-04-29,2013-05-03,2342,s,terracotta,5,69.90,91920,Mrs,1962-03-08,Bayern,2013-04-29,NaN,4,2013-04-29_91920,51,0,0,4,4,5,29,3,18,18,9,748.2,83.13,1,8,5,4,3,6,4,5,1,8,1,8,3,6,4,5,3,6,1,8,3,6
531166,50075,2013-04-29,2013-05-03,2505,s,terracotta,5,64.90,91920,Mrs,1962-03-08,Bayern,2013-04-29,NaN,4,2013-04-29_91920,51,0,0,4,4,5,29,3,18,18,9,748.2,83.13,2,7,5,4,3,6,4,5,2,7,2,7,3,6,4,5,3,6,2,7,3,6
531167,50076,2013-04-28,2013-05-02,2470,l,white,5,79.90,85095,Mrs,1950-02-14,Berlin,2013-03-24,NaN,4,2013-04-28_85095,63,35,6,3,4,5,28,2,17,18,1,79.9,79.90,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0
531168,50077,2013-04-28,2013-05-02,2452,m,white,5,59.90,91922,Mrs,1969-11-27,Bburg,2013-04-28,NaN,4,2013-04-28_91922,43,0,6,3,4,5,28,2,17,18,2,119.8,59.90,2,0,2,0,1,1,2,0,2,0,1,1,1,1,2,0,1,1,1,1,1,1


In [4]:
low_cardinality_cat_cols = ["size","item_color","user_title","user_state"]
high_cardinality_cat_cols = ["item_id","brand_id","user_id"]
cat_cols = low_cardinality_cat_cols + high_cardinality_cat_cols

In [35]:
# size of the known dataset
k = max(df["order_item_id"])-1
# Get the validation set
df_valid = df.iloc[k+1:, :]

In [36]:
# Get list of the months to train, remove months on which to test
months_to_train = list(range(1,13))
months_to_train.remove(3)
# Split into train and test. "~" in front of a variable means "not"
df_train = df.loc[:k][df.loc[:k,"order_month"].isin(months_to_train)]
df_test = df.loc[:k][~df.loc[:k,"order_month"].isin(months_to_train)]

# Drop unnecessary columns
columns_to_drop = ["order_date", "delivery_date", "user_dob", "user_reg_date", "order_id","order_item_id"]
df_train.drop(columns_to_drop, axis=1, inplace=True)
df_test.drop(columns_to_drop, axis=1, inplace=True)
df_valid.drop(columns_to_drop, axis=1, inplace=True)

Since we are no longer using one-hot encoding, we will rework our models not to include the one-hot encoding.

In [71]:
# Xgboost classifier to train the model and use ROC as the metric
def xgboost_classifier(df_train, df_test):
    X_train, Y_train = df_train.drop(["return"],axis=1), df_train["return"]
    X_test, Y_test = df_test.drop(["return"],axis=1), df_test["return"]
    
    model = XGBClassifier(max_depth=7, n_estimators=50, learning_rate=0.1,
                            n_jobs=-1, random_state=42)
    model.fit(X_train, Y_train, eval_metric='auc', eval_set=[(X_test, Y_test)],early_stopping_rounds = 20,verbose = 0)
    Y_pred = model.predict(X_test)
    auc = roc_auc_score(Y_test, Y_pred)
    print("Xgboost AUC on test set:", auc)
    return model,auc

In [9]:
# Catboost classifier to train the model and use ROC as the metric
def catboost_classifier(df_train, df_test,cat_cols):
    X_train, Y_train = df_train.drop(["return"],axis=1), df_train["return"]
    X_test, Y_test = df_test.drop(["return"],axis=1), df_test["return"]
    model = CatBoostClassifier(iterations=50, learning_rate=0.1, depth=10,
                            loss_function='Logloss', eval_metric='AUC',
                            random_seed=42)
    pool_train = Pool(X_train, Y_train,cat_features = cat_cols)

    pool_test = Pool(X_test, Y_test,cat_features = cat_cols)
    

    
    model.fit(pool_train, eval_set=pool_test, use_best_model=True, verbose=False)
    Y_pred = model.predict(pool_test)
    auc = roc_auc_score(Y_test, Y_pred)
    print("Catboost AUC on test set:", auc)
    return model,auc

In [70]:
# LightGBM
def lightgbm(df_train,df_test):
    X_train, Y_train = df_train.drop(["return"],axis=1), df_train["return"]
    X_test, Y_test = df_test.drop(["return"],axis=1), df_test["return"]
    
    model = LGBMClassifier(boosting_type='gbdt')
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)
    auc = roc_auc_score(Y_test, Y_pred)
    print("LGBM AUC on test set: ", auc)
    return model,auc

In [33]:
def neural_network(df_train,df_test,epochs):
    X_train, Y_train = df_train.drop(["return"],axis=1), df_train["return"]
    X_test, Y_test = df_test.drop(["return"],axis=1), df_test["return"]
    X_train = StandardScaler().fit_transform(X_train)
    X_test = StandardScaler().fit_transform(X_test)
    
    model = Sequential()
    model.add(Dense(units=64, activation='relu', input_dim=X_train.shape[1]))
    model.add(Dense(units=64, activation='relu'))
    model.add(Dense(units=1, activation='sigmoid'))
    auc = tensorflow.keras.metrics.AUC()
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[auc])
    results = model.fit(X_train, Y_train, epochs=epochs, batch_size=256, verbose=1,validation_data=(X_test, Y_test))
    Y_pred = model.predict(X_test)
    auc_1 = roc_auc_score(Y_test, Y_pred)
    print("ROC on test set:", auc_1)
    # print("MAE on test set:", mean_absolute_error(Y_test, Y_pred))
    return model,auc_1, results.history[auc.name], results.history['val_'+auc.name], Y_pred

In [37]:
# Encode categorical variables using leave one out encoding
def encode_categorical_columns(df_train,df_test,columns,sig):
    encoder = LeaveOneOutEncoder(cols=columns, return_df=True,sigma=sig, verbose=True)
    df_encode_train = encoder.fit_transform(df_train.drop(["return"],axis=1),df_train[["return"]])
    df_encode_test = encoder.transform(df_test.drop(["return"],axis=1))
    df_encode_train , df_encode_test = df_encode_train.join(df_train[["return"]]), df_encode_test.join(df_test[["return"]])
    return df_encode_train, df_encode_test, encoder

In [38]:
df_train, df_test, encoder = encode_categorical_columns(df_train,df_test,cat_cols,0.1)

In [93]:
df_train

,item_id,size,item_color,brand_id,item_price,user_id,user_title,user_state,delivery_time,user_age,user_reg_age,order_weekday,delivery_weekday,order_month,delivery_month,order_day,delivery_day,order_week,delivery_week,average_item_price_order,order_item_count,order_sum,order_number_same_item_id,order_number_different_item_id,order_number_same_size,order_number_different_size,order_number_same_item_color,order_number_different_item_color,order_number_same_brand_id,order_number_different_brand_id,order_number_same_item_id_size,order_number_different_item_id_size,order_number_same_item_id_item_color,order_number_different_item_id_item_color,order_number_same_size_item_color,order_number_different_size_item_color,order_number_same_size_brand_id,order_number_different_size_brand_id,order_number_same_item_color_brand_id,order_number_different_item_color_brand_id,order_number_same_item_id_size_item_color,order_number_different_item_id_size_item_color,order_number_same_size_item_color_brand_id,order_number_different_size_item_color_brand_id,return
0,0.617373,0.548014,0.520261,0.735728,69.90,0.539592,0.487594,0.365430,2,47,342,6,1,4,4,1,3,13,14,69.93,3,209.80,0,3,0,3,0,3,0,3,0,3,0,3,0,3,0,3,0,3,0,3,0,3,0.0
1,0.723669,0.453842,0.451558,0.443151,69.95,0.441709,0.483240,0.518341,2,47,342,6,1,4,4,1,3,13,14,69.93,3,209.80,1,2,1,2,0,3,1,2,1,2,0,3,0,3,1,2,0,3,0,3,0,3,1.0
2,0.619180,0.345758,0.486067,0.640610,69.95,0.446865,0.491034,0.455364,2,47,342,6,1,4,4,1,3,13,14,69.93,3,209.80,1,2,1,2,0,3,1,2,1,2,0,3,0,3,1,2,0,3,0,3,0,3,1.0
3,0.396502,0.490705,0.393220,0.658324,39.90,0.510317,0.404429,0.537943,4,52,89,0,4,4,4,2,6,14,14,39.90,1,39.90,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0.0
4,0.577156,0.550700,0.627206,0.717851,29.90,0.000000,0.461461,0.372865,4,48,411,0,4,4,4,2,6,14,14,83.23,3,249.70,0,3,1,2,1,2,0,3,0,3,0,3,1,2,0,3,0,3,0,3,0,3,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412004,0.579990,0.540648,0.509167,0.595892,69.95,0.944907,0.514203,0.500607,5,32,0,3,1,2,3,28,5,9,10,41.91,5,209.55,0,5,0,5,2,3,0,5,0,5,0,5,0,5,0,5,0,5,0,5,0,5,1.0
412005,0.548777,0.560873,0.447566,0.456449,34.90,1.083667,0.520958,0.576596,5,32,0,3,1,2,3,28,5,9,10,41.91,5,209.55,3,2,3,2,2,3,3,2,3,2,1,4,1,4,3,2,1,4,1,4,1,4,1.0
412006,0.623129,0.462269,0.508073,0.472124,34.90,1.009745,0.519297,0.549838,5,32,0,3,1,2,3,28,5,9,10,41.91,5,209.55,3,2,3,2,2,3,3,2,3,2,1,4,1,4,3,2,1,4,1,4,1,4,1.0
412007,0.636949,0.447757,0.487433,0.448513,34.90,1.039730,0.521708,0.474789,5,32,0,3,1,2,3,28,5,9,10,41.91,5,209.55,3,2,3,2,1,4,3,2,3,2,1,4,1,4,3,2,1,4,1,4,1,4,1.0


In [55]:
catboost_model,catboost_auc = catboost_classifier(df_train,df_test,[])

Catboost AUC on test set: 0.6625998718859887


In [56]:
lightgbm_model,lightgbm_auc = lightgbm(df_train,df_test)

LGBM AUC on test set:  0.6650706916029628


In [57]:
xgb_model,xgb_auc = xgboost_classifier(df_train,df_test)

Xgboost AUC on test set: 0.6622517664192289


In [39]:
neural_network_model, neural_network_auc, results_train, results_test, Y_pred = neural_network(df_train,df_test,10)

Epoch 1/10
1608/1608 [==============================] - 3s 2ms/step - loss: 0.5927 - auc_10: 0.7460 - val_loss: 0.6419 - val_auc_10: 0.6968
Epoch 2/10
1608/1608 [==============================] - 3s 2ms/step - loss: 0.5798 - auc_10: 0.7596 - val_loss: 0.6389 - val_auc_10: 0.7077
Epoch 3/10
1608/1608 [==============================] - 3s 2ms/step - loss: 0.5753 - auc_10: 0.7641 - val_loss: 0.6410 - val_auc_10: 0.7090
Epoch 4/10
1608/1608 [==============================] - 2s 2ms/step - loss: 0.5732 - auc_10: 0.7664 - val_loss: 0.6433 - val_auc_10: 0.7103
Epoch 5/10
1608/1608 [==============================] - 3s 2ms/step - loss: 0.5716 - auc_10: 0.7680 - val_loss: 0.6383 - val_auc_10: 0.7137
Epoch 6/10
1608/1608 [==============================] - 3s 2ms/step - loss: 0.5705 - auc_10: 0.7691 - val_loss: 0.6388 - val_auc_10: 0.7131
Epoch 7/10
1608/1608 [==============================] - 3s 2ms/step - loss: 0.5695 - auc_10: 0.7702 - val_loss: 0.6375 - val_auc_10: 0.7146
Epoch 8/10
1608/1608

In [61]:
auc_dict_target = {"Catboost":catboost_auc,"Xgboost":xgb_auc,"Lightgbm":lightgbm_auc,"Neural_Network":neural_network_auc}
model_results_target = pd.DataFrame(auc_dict_target,index=["AUC_encoding"])
model_results_target.sort_values(by="AUC_encoding",ascending=False,inplace=True,axis=1)
model_results_target = round(model_results_target.transpose(),3) 
model_results_target  

,AUC_encoding
Neural_Network,0.716
Lightgbm,0.665
Catboost,0.663
Xgboost,0.662


We can see that the tree-based models do not perform as good as the neural network model and are not affected by the encoding as their performances did not change. Overall using the **Leave-One-Out Encoding**, the neural network model performs better than before and reached a relatively good AUC score of **0.72**. From now on we are only going to focus on the neural network model.

In the next step, we will load the true labels of the data we have to predict from **orders_realclass.txt** and test our model on the data from 2013-04-01 to 2013-04-30.

In [9]:
# Get the real class predictions for the test set
real_class = pd.read_csv("orders_realclass.txt", delimiter=";")
real_class = real_class.astype({"orderItemID":int,"returnShipment":int})
df_valid.reset_index(drop=True,inplace=True)
df_valid["return"] = real_class["returnShipment"]

In [12]:
# Combine the training and former testing sets into one dataframe
df_train_new = pd.concat([df_train,df_test],axis=0)
# Encoding the categorical variables
df_train_encoded, df_test_encoded, encoder = encode_categorical_columns(df_train_new,df_valid,cat_cols,0.1)

In [32]:
neural_network_model, neural_network_auc, results_train, results_test,Y_probability = neural_network(df_train_encoded,df_test_encoded,6)

Epoch 1/6
1880/1880 [==============================] - 3s 2ms/step - loss: 0.5915 - auc_9: 0.7474 - val_loss: 0.6282 - val_auc_9: 0.7136
Epoch 2/6
1880/1880 [==============================] - 3s 1ms/step - loss: 0.5784 - auc_9: 0.7612 - val_loss: 0.6271 - val_auc_9: 0.7190
Epoch 3/6
1880/1880 [==============================] - 3s 1ms/step - loss: 0.5750 - auc_9: 0.7647 - val_loss: 0.6253 - val_auc_9: 0.7249
Epoch 4/6
1880/1880 [==============================] - 3s 1ms/step - loss: 0.5733 - auc_9: 0.7664 - val_loss: 0.6240 - val_auc_9: 0.7226
Epoch 5/6
1880/1880 [==============================] - 3s 2ms/step - loss: 0.5719 - auc_9: 0.7679 - val_loss: 0.6229 - val_auc_9: 0.7246
Epoch 6/6
1880/1880 [==============================] - 3s 1ms/step - loss: 0.5707 - auc_9: 0.7691 - val_loss: 0.6275 - val_auc_9: 0.7278
ROC on test set: 0.727791030080463


Our model reached a score of **0.73** indicating a slight increase in AUC compared to the previous model which didn't include the Month with missing return values. This increase could be explained by the fact that the model we previously trained included 12 Months of transactional time dependant data and the new data pushed our timetable to 13 Months. Thus we have 1 Month that repeats itself (April) and the model can pick up the trends in the data more easily.